In [6]:
from datasets import DatasetDict, Dataset

output_dataset = ".storage/data/smcq_dataset4"

reframed_questions = f"{output_dataset}_{{}}.csv"

reframed_dataset = {}

splits = ["train", "validation", "test"]
for split in splits:
  reframed_dataset[split] = Dataset.from_csv(reframed_questions.format(split))

reframed_dataset = DatasetDict(reframed_dataset)
reframed_dataset

DatasetDict({
    train: Dataset({
        features: ['index', 'prompt', 'reframed_question', 'question', 'answer', 'label'],
        num_rows: 70000
    })
    validation: Dataset({
        features: ['index', 'prompt', 'reframed_question', 'question', 'answer', 'label'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['index', 'prompt', 'reframed_question', 'question', 'answer', 'label'],
        num_rows: 60
    })
})

In [7]:
import re

r_qst_pat = re.compile(r'F:\s*"(.+)"$')

def parse_question(example):
  return r_qst_pat.search(example["question"]) is not None

for split in splits:
  reframed_dataset[split] = reframed_dataset[split]\
    .remove_columns(["index", "prompt", "question"])\
    .rename_column("reframed_question", "question")\
    .filter(parse_question, batched=False)\
    .map(lambda x: {**x, "question": r_qst_pat.search(x["question"]).group(1)}, batched=False)
  
reframed_dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'label'],
        num_rows: 69083
    })
    validation: Dataset({
        features: ['question', 'answer', 'label'],
        num_rows: 4929
    })
    test: Dataset({
        features: ['question', 'answer', 'label'],
        num_rows: 47
    })
})

In [9]:
reframed_dataset.save_to_disk(output_dataset)

output_splits = f"{output_dataset}/{{}}.csv"

for split in splits:
  reframed_dataset[split].to_csv(output_splits.format(split))

Creating CSV from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 1034.35ba/s]
